In [ ]:
##Write a python program to display all the header tags from wikipedia.org
from urllib.request import urlopen
from bs4 import BeautifulSoup
html = urlopen('https://en.wikipedia.org/wiki/Main_Page')
bs = BeautifulSoup(html, "html.parser")
titles = bs.find_all(['h1', 'h2','h3','h4','h5','h6'])
print('List all the header tags :', *titles, sep='\n\n')

In [ ]:
##Write a python program to display IMDB’s Top rated 100 movies’ data (i.e. name, rating, year of release) and make data frame.
from bs4 import BeautifulSoup
import requests
import random
url = 'http://www.imdb.com/chart/top100'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
def get_imd_movies(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    movies = soup.find_all("td", class_="titleColumn")
    random.shuffle(movies)
    return movies
def get_imd_movie_info(movie):
    movie_title = movie.a.contents[0]
    movie_year = movie.span.contents[0]
    movie_url = 'http://www.imdb.com' + movie.a['href']
    return movie_title, movie_year, movie_url

def imd_movie_picker():
    ctr=0
    print("--------------------------------------------")
    for movie in get_imd_movies('http://www.imdb.com/chart/top'):
        movie_title, movie_year, movie_url = get_imd_movie_info(movie)
        print(movie_title, movie_year)
        print("--------------------------------------------")
        ctr=ctr+1
        if (ctr==100):
          break;   
if __name__ == '__main__':
    imd_movie_picker()

In [ ]:
##Write a python program to display IMDB’s Top rated 100 indian movies’ data (i.e. name, rating, year of release) and make data frame.
from bs4 import BeautifulSoup
import requests
import random
url = 'https://www.imdb.com/india/top-rated-indian-movies/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
def get_imd_movies(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    movies = soup.find_all("td", class_="titleColumn")
    random.shuffle(movies)
    return movies
def get_imd_movie_info(movie):
    movie_title = movie.a.contents[0]
    movie_year = movie.span.contents[0]
    movie_url = 'http://www.imdb.com' + movie.a['href']
    return movie_title, movie_year, movie_url

def imd_movie_picker():
    ctr=0
    print("--------------------------------------------")
    for movie in get_imd_movies('https://www.imdb.com/india/top-rated-indian-movies/'):
        movie_title, movie_year, movie_url = get_imd_movie_info(movie)
        print(movie_title, movie_year)
        print("--------------------------------------------")
        ctr=ctr+1
        if (ctr==100):
          break;   
if __name__ == '__main__':
    imd_movie_picker()

In [ ]:
##5) Write a python program to scrape cricket rankings from icc-cricket.com. You have to scrape:
#a) Top 10 ODI teams in men’s cricket along with the records for matches, points and rating.
#b) Top 10 ODI Batsmen along with the records of their team and rating.
#c) Top 10 ODI bowlers along with the records of their team and rating.
#6) Write a python program to scrape cricket rankings from icc-cricket.com. You have to scrape:
#a) Top 10 ODI teams in women’s cricket along with the records for matches, points and rating.
#b) Top 10 women’s ODI Batting players along with the records of their team and rating.
#c) Top 10 women’s ODI all-rounder along with the records of their team and rating.
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"
    }

urls = ["https://www.icc-cricket.com/rankings/mens/team-rankings/odi",
"https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting",
"https://www.icc-cricket.com/rankings/mens/player-rankings/odi/bowling",
"https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting",
"https://www.icc-cricket.com/rankings/womens/team-rankings/odi",
"https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting",
"https://www.icc-cricket.com/rankings/womens/player-rankings/odi/bowling",]
final_result_file_name = "All Ranking List.csv"
final_column_names = ["Ranking Type", "Position", "Player Name", "Team Name", "Rating", "Career Best Rating", "Crawl URL"]
pd.DataFrame(columns=final_column_names).to_csv(final_result_file_name, sep="\t", index=False, encoding="utf-8")

for url in urls:
    request_object = requests.get(url, headers=headers)
    html_content = request_object.text
    print(request_object.status_code, "->", url)
    soup_object = BeautifulSoup(html_content, "lxml")
    for element in soup_object.select('[class="ranking-pos up"], [class="ranking-pos down"]'):
        element.replace_with(BeautifulSoup("<" + element.name + "></" + element.name + ">", "html.parser"))

    ranking_type = soup_object.select_one(".rankings-block__title-container > h4").text

    result_file_name = ranking_type + ".csv"
    column_names = ["Position", "Player Name", "Team Name", "Rating", "Career Best Rating", "Crawl URL"]
    pd.DataFrame(columns=column_names).to_csv(result_file_name, sep="\t", index=False, encoding="utf-8")

    for element in soup_object.select('table[class="table rankings-table"] tr'):
        if(element.find("th")):
            continue
        data_dict = dict()
        data_dict["Crawl URL"] = url
        data_dict["Ranking Type"] = ranking_type
        if(element.select_one('[class*="position"]')):
            data_dict["Position"] = element.select_one('[class*="position"]').text
        for player_name in (element.select('a[href*="/player-rankings"]')):
            if(player_name.text.strip()):
                data_dict["Player Name"] = player_name.text
        if(element.select_one('[class^="flag-15"]')):
            data_dict["Team Name"] = element.select_one('[class^="flag-15"]')["class"][-1]
        if(element.select_one('[class$="rating"]')):
            data_dict["Rating"] = element.select_one('[class$="rating"]').text
        if(element.select_one('td.u-hide-phablet')):
            data_dict["Career Best Rating"] = element.select_one('td.u-hide-phablet').text
        for key in data_dict.keys():
            data_dict[key] = re.sub(r"\s+", " ", data_dict[key])
            data_dict[key] = data_dict[key].strip()
        pd.DataFrame([data_dict], columns=column_names).to_csv(result_file_name, sep="\t", index=False, header=False, encoding="utf-8", mode="a")
        pd.DataFrame([data_dict], columns=final_column_names).to_csv(final_result_file_name, sep="\t", index=False, header=False, encoding="utf-8", mode="a")

In [ ]:
from bs4 import BeautifulSoup
import requests


In [ ]:
#Write a python program to scrape mentioned details from dineout.co.in :
#i) Restaurant name
#ii) Cuisine
#iii) Location
#iv) Ratings
#v) Image URL
page=requests.get('https://www.dineout.co.in/delhi-restaurants/buffet-special')
page

In [ ]:
soup=BeautifulSoup(page.content)
soup

In [ ]:
first_title=soup.find('div',class_="restnt-info cursor")
first_title

In [ ]:
titles=[]
for i in soup.find_all('div',class_="restnt-info cursor"):
    titles.append(i.text)
    
titles

In [ ]:
cuisines=[]
for i in soup.find_all('span',class_="double-line-ellipsis"):
    cuisines.append(i.text)
    
cuisines

In [ ]:
location=[]
for i in soup.find_all('div',class_="restnt-loc ellipsis"):
    location.append(i.text)
location

In [ ]:
ratings=[]
for i in soup.find_all('div',class_="restnt-rating rating-4"):
    ratings.append(i.text)
ratings

In [ ]:
images_url=[]
for i in soup.find_all('img',class_="no-img"):
    images_url.append(i["data-src"])
images_url

In [ ]:
import pandas as pd
df=pd.DataFrame({'Titles':titles,'Cuisines':cuisines,'Location':location,'Ratings':ratings,'Images_url':images_url})
df

In [ ]:
#Write a python program to scrape first 10 product details which include product name , price , Image URL from https://www.bewakoof.com/women-tshirts?ga_q=tshirts .
page=requests.get('https://www.bewakoof.com/women-tshirts?ga_q=tshirts')
soup1=BeautifulSoup(page.content)
productnames=[]
for i in soup1('div',class_="productCardDetail "):
    productnames.append(i.text)
prices=[]
for i in soup1('span',class_="discountedPriceText "):
    prices.append(i.text)
images_url=[]
for i in soup1('img',class_="productImgTag"):
    images_url.append(i["src"])
import pandas as pd
df1=pd.DataFrame({'Product_Names':productnames,'Prices':prices,'Images_url':images_url})
df1

In [ ]:
#Write a python program to scrape product name, price and discounts from https://meesho.com/bags-ladies/pl/p7vbp .
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
page=requests.get('https://meesho.com/bags-ladies/pl/p7vbp')
soup2=BeautifulSoup(page.content)
productnames1=[]
for i in soup2('div',class_="Text__StyledText-sc-oo0kvp-0 bWSOET NewProductCard__ProductTitle_Desktop-sc-j0e7tu-4 cQhePS NewProductCard__ProductTitle_Desktop-sc-j0e7tu-4 cQhePS"):
    productnames1.append(i.text)
prices1=[]
for i in soup2('div',class_="Card__BaseCard-sc-b3n78k-0 iLPHgK NewProductCard__PriceRow-sc-j0e7tu-5 eyya-Dr NewProductCard__PriceRow-sc-j0e7tu-5 eyya-Dr"):
    prices1.append(i.text)
discounts=[]
for i in soup2('img',class_="Card__BaseCard-sc-b3n78k-0 dpAIfg NewProductCard__BadgeRow-sc-j0e7tu-13 ezFwfg NewProductCard__BadgeRow-sc-j0e7tu-13 ezFwfg"):
    discounts.append(i.text)
df2=pd.DataFrame({'Product_Names':1,'Prices':prices1,'Discounts':discounts})
df2